In [1]:
%cd ../../../../

/home/users/dmoreno2016/ASTROMER/astromer_pe_main/astromer


In [2]:
import pandas as pd
import numpy as np

import toml
import yaml

import glob

In [3]:
path_exp_files = 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean'
path_exp_files

'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean'

In [4]:
path_exps = glob.glob('{}/*'.format(path_exp_files))
path_exps

['presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean/exp_005_a',
 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean/exp_004_a',
 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean/exp_000_a_sampling_1',
 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean/exp_001_a',
 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean/exp_003_a',
 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean/exp_000_a_all_windows_1',
 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean/exp_000_a',
 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean/exp_000_b']

In [23]:
folds = [0, 1, 2]

#ids = []
#config = []
#pe_config = []
#test_metrics = []
#test_metrics_datasets = []

df_results = []
dict_results = {}
for path_exp in path_exps:
    if path_exp.split('/')[-1].find('exp_000_a_') == -1:
        ids = {'ID': path_exp.split('/')[-1]}

        file = open('{}/config.yaml'.format(path_exp), "r")
        config = yaml.load(file, Loader=yaml.FullLoader) 

        file = open('{}/pe_config.yaml'.format(path_exp), "r")
        pe_config = yaml.load(file, Loader=yaml.FullLoader)
        pe_config = {key: [value] for key, value in pe_config[list(pe_config.keys())[0]].items()}

        with open('./{}/test_metrics.toml'.format(path_exp), mode="r") as f:
            test_metrics = toml.load(f)

        dict_metrics = test_metrics['macho_clean']
        dict_stat_metrics = dict()
        r2 = []
        for fold in folds:
            r2.append(dict_metrics['fold_{}'.format(fold)]['r_square'])
        dict_stat_metrics['r2_mean'] = np.mean(r2)
        dict_stat_metrics['r2_std'] = np.std(r2)

        dict_results.update(ids)
        dict_results.update(config)
        dict_results.update(pe_config)
        dict_results.update(dict_stat_metrics)

        #print(dict_results)
        df_results.append(pd.DataFrame.from_dict(dict_results))

        #
        # 
        # with open('./{}/test_metrics_datasets_50.toml'.format(path_exp), mode="r") as f:
        #    test_metrics_datasets.append(toml.load(f))

df_results = pd.concat(df_results).reset_index(drop=True)
df_results

,ID,avg_layers,bs,checkpoint,data,debug,dropout,encoder_mode,exp_name,gpu,...,r2_mean,r2_std,H,T_max,initializer,pe_trainable,base,c,max_period,min_period
0,exp_005_a,False,2500,-1,data/records/macho_clean,False,0.0,normal,exp_005_a,0,...,0.738996,0.000118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,exp_004_a,False,2500,-1,data/records/macho_clean,False,0.0,normal,exp_004_a,0,...,0.728421,0.000225,64.0,1500.0,random_normal,True,NaN,NaN,NaN,NaN
2,exp_001_a,False,2500,-1,./data/records/macho_clean,False,0.0,normal,exp_001_a,0,...,0.751565,0.000024,64.0,1500.0,pe_astromer,False,1000.0,1.0,1825.0,0.01
3,exp_003_a,False,2500,-1,./data/records/macho_clean,False,0.0,normal,exp_003_a,1,...,0.583527,0.000238,64.0,1500.0,pe_astromer,False,1000.0,1.0,1825.0,0.01
4,exp_000_a,False,2500,-1,data/records/macho_clean,False,0.0,normal,exp_000_a,0,...,0.762443,0.000393,64.0,1500.0,pe_astromer,False,1000.0,1.0,1825.0,0.01
5,exp_000_b,False,2500,-1,data/records/macho_clean,False,0.0,normal,exp_000_b,0,...,0.729508,0.000236,64.0,1500.0,pe_astromer,True,1000.0,1.0,1825.0,0.01


In [24]:
df_results.columns

Index(['ID', 'avg_layers', 'bs', 'checkpoint', 'data', 'debug', 'dropout',
       'encoder_mode', 'exp_name', 'gpu', 'head_dim', 'lr', 'mixer',
       'num_epochs', 'num_heads', 'num_layers', 'patience', 'pe_func_name',
       'pe_type', 'probed', 'pt_folder', 'repeat', 'residual_type', 'rs',
       'sampling', 'window_size', 'm_layers', 'use_mjd', 'r2_mean', 'r2_std',
       'H', 'T_max', 'initializer', 'pe_trainable', 'base', 'c', 'max_period',
       'min_period'],
      dtype='object')

In [25]:
df_results[['ID', 'pe_type', 'pe_func_name', 'r2_mean', 'r2_std']]

,ID,pe_type,pe_func_name,r2_mean,r2_std
0,exp_005_a,APE,pe_mlp,0.738996,0.000118
1,exp_004_a,APE,pe_tm,0.728421,0.000225
2,exp_001_a,APE,pe,0.751565,0.000024
3,exp_003_a,MixPE,pe,0.583527,0.000238
4,exp_000_a,APE,pe,0.762443,0.000393
5,exp_000_b,APE,pe,0.729508,0.000236


In [54]:
folds = [0, 1, 2]

In [57]:
dict_metrics = test_metrics[0]['macho_clean']
dict_stat_metrics = dict()

r2 = []
for fold in folds:
    r2.append(dict_metrics['fold_{}'.format(fold)]['r_square'])

dict_stat_metrics['r2_mean'] = np.mean(r2)
dict_stat_metrics['r2_std'] = np.std(r2)

dict_stat_metrics

{'r2_mean': 0.738996167977651, 'r2_std': 0.00011758959992207549}

In [51]:
for name_data in test_metrics_datasets[0].keys():

    dict_metrics = test_metrics_datasets[0][name_data]

    r2 = []
    for fold in folds:
        r2.append(dict_metrics['fold_{}'.format(fold)]['r_square'])

    dict_stat_metrics['r2_mean'] = np.mean(r2)
    dict_stat_metrics['r2_std'] = np.std(r2)
    
    break

alcock


In [53]:
np.std(r2)
np.mean(r2)

0.5668821930885315

In [ ]:
pd.DataFrame.from_dict({key: [value] for key, value in config[0].items()})

In [20]:
ids

['exp_005_a',
 'exp_004_a',
 'exp_000_a_sampling_1',
 'exp_001_a',
 'exp_003_a',
 'exp_000_a_all_windows_1',
 'exp_000_a',
 'exp_000_b']

In [17]:
test_metrics

[{'macho_clean': {'fold_0': {'loss': 0.08845061808824539,
    'r_square': 0.7389798164367676,
    'rmse': 0.08845061808824539},
   'fold_1': {'loss': 0.08855785429477692,
    'r_square': 0.7388610243797302,
    'rmse': 0.08855785429477692},
   'fold_2': {'loss': 0.0883399099111557,
    'r_square': 0.7391476631164551,
    'rmse': 0.0883399099111557}}},
 {'macho_clean': {'fold_0': {'loss': 0.08999898284673691,
    'r_square': 0.7287216186523438,
    'rmse': 0.08999898284673691},
   'fold_1': {'loss': 0.09005183726549149,
    'r_square': 0.7283596992492676,
    'rmse': 0.09005183726549149},
   'fold_2': {'loss': 0.09008192270994186,
    'r_square': 0.7281802892684937,
    'rmse': 0.09008192270994186}}},
 {'macho_clean': {'fold_0': {'loss': 0.08665117621421814,
    'r_square': 0.7514470815658569,
    'rmse': 0.08665117621421814},
   'fold_1': {'loss': 0.08668816089630127,
    'r_square': 0.750964343547821,
    'rmse': 0.08668816089630127},
   'fold_2': {'loss': 0.0867423564195633,
    'r_s